# MIT 6.036 Homework 8 - Fall 2021

#Part 3: Augmentations

### Goals of this Assigment:

* Practice implementing data augmentations

* Learn the basics of PyTorch, one of the most popular Python frameworks for implementing neural networks

### Introduction
In the past, we've said that one way to combat overfitting on our training set is to simply collect more training data. However, sometimes we have to train a model in low data regimes, and we find our model overfits to the training set too easily and performs poorly on unseen data. One way to increase our training set size is to make up new data! In particular, we do this using transformations on the original data. For every example pair $(x^{(i)}, y^{(i)})$ in our dataset, we generate a new pair of data points $({x^{(i)}}^{\text{aug}}, y_i)$, where our label is unchanged but our input has been transformed in some way that still preserves a human's ability to predict it.


---

# **RUN THIS NOTEBOOK WITH GPU**

OTHERWISE YOU WILL SPEND A LONG TIME ON TRAINING

**To do this, go to `Runtime` > `Change runtime type` and under `hardware acceleration` set the dropdown to `GPU`.**


# Problem 0: Boilerplate

**You do NOT need to change anything in the code boxes in this section.**

Please run each code box to get set up. You may find reviewing this code helpful for figuring out how to implement the later sections.

In [ ]:
## RUN ME! No changes necessary
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import torchvision
import cv2
import torch
from torch import nn
import random
from tqdm import tqdm


!rm code_for_hw08.zip
!wget --no-check-certificate --quiet https://go.odl.mit.edu/subject/6.036/_static/catsoop/homework/hw08/code_for_hw08.zip
!unzip -q code_for_hw08.zip -d code_for_hw08
!mv code_for_hw08/* .

def custom_pil_loader(path):
# open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        img.load()
        return img

rm: cannot remove 'code_for_hw08.zip': No such file or directory


In [ ]:
## RUN ME! No changes necessary
def get_datasets(train_transforms, test_transforms):
  train_data = FashionMNIST("./fashion_mnist", train=True, download=True, transform=train_transforms)
  test_data = torchvision.datasets.ImageFolder("test_images", transform=test_transforms, loader=custom_pil_loader)
  train_loader = torch.utils.data.DataLoader(train_data, 10, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_data, 10, shuffle=False)
  return train_loader, test_loader

# Problem 0.1

Copy and paste your `train` and `test` functions from a previous notebook here. **Note: Which one should you use? The notebook 1 and notebook 2 functions are different**


In [ ]:
# EDIT ME!

def train(model, device, train_loader, optimizer, val_loader=None, pbar=False):
    '''
    Function for training our networks. One call to train() performs a single
    epoch for training.

    model: an instance of our model, in this assignment, this will be your autoencoder

    device: either "cpu" or "cuda", depending on if you're running with GPU support

    train_loader: the dataloader for the training set

    optimizer: optimizer used for training (the optimizer implements SGD)

    val_loader: (optional) validation set to include
    '''

    # Set the model to training mode.
    model.train()

    #we'll keep adding the loss of each batch to total_loss, so we can calculate
    #the average loss at the end of the epoch.
    total_loss = 0
    total_correct = 0
    total_items = 0

    # We'll iterate through each batch. One call of train() trains for 1 epoch.
    # batch_idx: an integer representing which batch number we're on
    # input: a pytorch tensor representing a batch of input images.
    if pbar:
      train_loader = tqdm(train_loader)

    for batch_idx, (input,target) in enumerate(train_loader):
        # This line sends data to GPU if you're using a GPU
        input = input.to(device)
        target = target.type(torch.LongTensor).to(device)

        # initialze the optimizer (the optimizer implements SGD)
        optimizer.zero_grad()

        # feed our input through the network
        output = model.forward(input)

        ## TODO: YOUR CODE HERE

        loss_function = nn.CrossEntropyLoss()
        loss_value = loss_function(output, target)

        ## END YOUR CODE

        # Perform backprop
        loss_value.backward()
        optimizer.step()

        #accumulate loss to later calculate the average
        total_loss += loss_value
        total_correct += torch.sum( torch.argmax(output, dim=1)==target )
        total_items += input.shape[0]


    return total_loss.item()/len(train_loader), (total_correct/total_items).item()

In [ ]:
# EDIT ME!
def test(model, device, test_loader, pbar=False):
    '''
    Function for testing our models. One call to test() runs through every
    datapoint in our dataset once.

    model: an instance of our model, in this assignment, this will be your autoencoder

    device: either "cpu" or "cuda:0", depending on if you're running with GPU support

    test_loader: the dataloader for the data to run the model on
    '''
    # set model to evaluation mode
    model.eval()

    # we'll keep track of total loss to calculate the average later
    test_loss = 0
    total_correct = 0
    total_items = 0

    #don't perform backprop if testing
    with torch.no_grad():
        # iterate thorugh each test image
        if pbar:
          test_loader = tqdm(test_loader)
        for (input,target) in test_loader:

            # send input image to GPU if using GPU
            input = input.to(device)
            target = target.type(torch.LongTensor).to(device)

            # run input through our model
            output = model(input)

            ## TODO: YOUR CODE HERE

            loss_function = nn.CrossEntropyLoss()
            loss_value = loss_function(output, target)

            ## END YOUR CODE

            # Accumulate for accuracy
            test_loss += loss_value
            total_correct += torch.sum( torch.argmax(output, dim=1)==target )
            total_items += input.shape[0]

    # calculate average loss/accuracy per batch
    test_loss /= len(test_loader)
    accuracy = total_correct / total_items

    return test_loss.item(), accuracy.item()

# Question 1

You are an engineer at a hot new e-commerce startup, AMLzon. AMLzon hopes to revolutionize e-commerce by applying computer vision to classify pictures of products. The CEO of AMLzon, Jeff BackPropezos gives you a training dataset of images from a fashion dataset. You validate your training dataset on images taken directly from customer images on the AMLzon e-commerce platform.

Run the code for training the neural network for three epochs without any additional augmentation. What is the value of the training and validation accuracy?


In [ ]:
# RUN ME! No changes necessary
epochs = 3

## DO NOT CHANGE THESE NEXT TWO LINES IF YOU WANT TO BE GRADED CORRECTLY
torch.manual_seed(0)
random.seed(0)

train_transforms = transforms.Compose([
  transforms.ToTensor(),
])
test_transforms = transforms.Compose([
  transforms.ToTensor(),
])

train_loader, val_loader = get_datasets(train_transforms, test_transforms)

# check if running on CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model =  torch.nn.Sequential(nn.Conv2d(1, 10, kernel_size=5),
                             nn.MaxPool2d(2),
                             nn.ReLU(),
                             nn.Conv2d(10, 20, kernel_size=5),
                             nn.MaxPool2d(2),
                             nn.ReLU(),
                             nn.Flatten(),
                             nn.Linear(320, 50),
                             nn.Linear(50, 10))
model.to(device)
# initialize our optimizer. We'll use Adam
optimizer = torch.optim.Adam(model.parameters())

# train your  classifier
for epoch in range(1, epochs+1):
    train_loss, train_acc = train(model, device, train_loader, optimizer, pbar=True)
    val_loss, val_acc = test(model, device, val_loader, pbar=True)
    print('Train Epoch: {:02d} \tTraining Loss: {:.6f} \tTraining Acc: {:.6f}\n \t\t\tValidation Loss: {:.6f} \tValidation Acc: {:.6f}\n'.format(epoch, train_loss, train_acc, val_loss, val_acc))

100%|██████████| 1000/1000 [00:04<00:00, 244.57it/s]


Train Epoch: 01 	Training Loss: 0.483003 	Training Acc: 0.823400
 			Validation Loss: 1.990938 	Validation Acc: 0.501200



100%|██████████| 1000/1000 [00:04<00:00, 248.74it/s]


Train Epoch: 02 	Training Loss: 0.337552 	Training Acc: 0.877850
 			Validation Loss: 2.555112 	Validation Acc: 0.479200



100%|██████████| 1000/1000 [00:03<00:00, 251.85it/s]

Train Epoch: 03 	Training Loss: 0.304921 	Training Acc: 0.888283
 			Validation Loss: 2.639710 	Validation Acc: 0.483500



# Question 2

After closely examining the validation dataset, you realize that merchants tend to randomly rotate images of their products between 45 degrees to the left and 45 degrees to the right! Use an augmentation to randomly rotate your training data between 45 degrees to the left and 45 degrees to the right.

**Hint:** Documentation <a href=https://pytorch.org/vision/stable/transforms.html> here</a> will be extremely useful!

In [ ]:
epochs = 3

## DO NOT CHANGE THESE NEXT TWO LINES IF YOU WANT TO BE GRADED CORRECTLY
torch.manual_seed(0)
random.seed(0)

train_transforms = transforms.Compose([
  transforms.ToTensor(),
  transforms.RandomRotation(45)
  # transforms.RandomRotation(random.choice(list(range(-45, 46)))),
])
test_transforms = transforms.Compose([
  transforms.ToTensor(),
])

train_loader, val_loader = get_datasets(train_transforms, test_transforms)

# check if running on CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model =  torch.nn.Sequential(nn.Conv2d(1, 10, kernel_size=5),
                             nn.MaxPool2d(2),
                             nn.ReLU(),
                             nn.Conv2d(10, 20, kernel_size=5),
                             nn.MaxPool2d(2),
                             nn.ReLU(),
                             nn.Flatten(),
                             nn.Linear(320, 50),
                             nn.Linear(50, 10))
model.to(device)
# initialize our optimizer. We'll use Adam
optimizer = torch.optim.Adam(model.parameters())

# train your  classifier
for epoch in range(1, epochs+1):
    train_loss, train_acc = train(model, device, train_loader, optimizer, pbar=True)
    val_loss, val_acc = test(model, device, val_loader, pbar=True)
    print('Train Epoch: {:02d} \tTraining Loss: {:.6f} \tTraining Acc: {:.6f}\n \t\t\tValidation Loss: {:.6f} \tValidation Acc: {:.6f}\n'.format(epoch, train_loss, train_acc, val_loss, val_acc))


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./fashion_mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./fashion_mnist/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./fashion_mnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./fashion_mnist/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

...

Train Epoch: 01 	Training Loss: 0.718459 	Training Acc: 0.732000
 			Validation Loss: 0.577282 	Validation Acc: 0.785300



100%|██████████| 1000/1000 [00:04<00:00, 222.97it/s]


Train Epoch: 02 	Training Loss: 0.538246 	Training Acc: 0.801617
 			Validation Loss: 0.524854 	Validation Acc: 0.803000



100%|██████████| 1000/1000 [00:04<00:00, 225.20it/s]

Train Epoch: 03 	Training Loss: 0.497402 	Training Acc: 0.815567
 			Validation Loss: 0.487894 	Validation Acc: 0.820300

